# Numerical Approximation Methods

The reason I'm doing this with Python is because I want to become more comfortable with the construction of many numerical approximation methods.

## Contents
1. [Initial Value Problems of the First Order](#Initial-Value-Problems-of-the-First-Order)
    1. [First Order Euler Method](#First-Order-Euler-Method)
    1. [Second Order Euler Method](#Second-Order-Euler-Method)
    1. [Runge-Kutta Method](#Runge-Kutta-Method)
    1. [Method Constructors](#Method-Constructors)
1. [Second Order ODE](#Second-Order-ODE)
    1. [Second Order Method Constructors](#Second-Order-Method-Constructors)
    1. [Gravity](#Gravity)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

## Initial Value Problems of the First Order
We will compare the accuracy of numerical approximation methods with varying timesteps.

In [ ]:
# create initial conditions
df = lambda t, x: - np.power(x, 2)
initial_value = 1

# create resolution space
nn = [i for i in range(2, 10)]
ntt = [np.linspace(0, 1, n) for n in nn]
nss = [[None for _ in tt] for tt in ntt]

# create different colors for each resolution
ncolor = cm.rainbow(np.linspace(0, 1, len(nn)))

### First Order Euler Method
Consider the differential equation $\dot{x(t)} = f(t, x(t))$. The first-order Euler's method with timestep $h$ is $x' = x + hf(t, x)$, where $x'$ is the approximated value of $x$ at the incremented time $t + h$.

We consider the autonomous differential equation $\dot{x} = -x^2$, with initial condition $x(0) = 1$. Therefore, the function in question is $f(t, x) = -x^2$.

In [ ]:
# iterate over different resolutions (using the first order Euler method).
for i, tt in enumerate(ntt):
    dt = (tt[-1] - tt[0]) / (len(tt) - 1)
    
    for j, t in enumerate(tt):
        if j == 0:
            nss[i][j] = initial_value
            pass
        else:
            # application of first-order euler method.
            nss[i][j] = nss[i][j-1] + df(t, nss[i][j-1])*dt
            pass
        pass
    plt.plot(tt, nss[i], label = '{}'.format(nn[i]), color = ncolor[i])
    pass
plt.ylim(0, 1.1)
plt.legend()
plt.show()

### Second Order Euler Method

Using Taylor's expansion, a second-order Euler approximation is $x' = x + h\dot{x} + \dfrac{h^2}{2}\ddot{x}$.

Since $\dot{x} = -x^2$, then $\ddot{x} = \dfrac{d}{dt}(-x^2) = -2x\dot{x} = -2x(-x^2) = 2x^3$. Let $g(t, x) = 2x^3$ be our second-order Euler function, then our approximation reduces to $x' = x - hx^2 + h^2x^3$.

In [ ]:
# define second-order term
ddf = lambda t, x:2*np.power(x, 3)

# iterate over different resolutions (using the second order Euler method).
for i, tt in enumerate(ntt):
    dt = (tt[-1] - tt[0]) / (len(tt) - 1)
    
    for j, t in enumerate(tt):
        if j == 0:
            nss[i][j] = initial_value
            pass
        else:
            # application of second-order euler method.
            nss[i][j] = nss[i][j-1] + df(t, nss[i][j-1])*dt + np.power(dt, 2)/2*ddf(t, nss[i][j-1]*dt)
            pass
        pass
    plt.plot(tt, nss[i], label = '{}'.format(nn[i]), color = ncolor[i])
    pass
plt.ylim(0, 1.1)
plt.legend()
plt.show()

### Runge-Kutta Method
The RK method states that $y' = y + \dfrac{1}{6}(k_1 + 2k_2 + 2k_3 + k_4)$, where $k_1 = hf(t_n, y_n)$, $k_2 = hf\bigg(t_n + \dfrac{h}{2}, y_n + \dfrac{k_1}{2}\bigg)$, $k_3 = hf\bigg(t_n + \dfrac{h}{2}, y_n + \dfrac{k_2}{2}\bigg)$, and $k_4 = hf(t_n + h, y_n + k_3)$.

### Method Constructors
We note that in all three approximation procedures, the method is applied in only one part of the algorithm. Hence, we can isolate the methods by defining their *constructors*, which take in all relevant functions and the timestep, and return *functions*, which take in only the current time and point in space. For our experiment, we compute all our derivatives by hand.

The purpose of writing constructors is to make as few iterated checks on the system as possible. For example, we don't need an iterated function which always checks whether a given value is an integer.

In [ ]:
from scipy.special import gamma

In [ ]:
def euler_method_constructor_first_order(ff):
    # computes using euler method of order equal to length of function list.
    # though we have higher orders in here, this constructor will not be used beyond this section.
    
    if not isinstance(ff, list):
        raise TypeError('ff must be list of functions.')
        pass
    
    # find order of ff.
    order = len(ff)
    
    # return a function which works for a given stepsize.
    def euler_method_stepsize_first_order(h):
        
        # return a function which, at a stepsize, returns next value.
        def euler_method_function_first_order(t, x):
            return x + sum(ff[i](t, x)*np.power(h, i + 1)/gamma(i + 1) for i in range(order))
        return euler_method_function_first_order
    return euler_method_stepsize_first_order

def runge_kutta_constructor_first_order(f):
    # constructs a function according to the rk4 algorithm.
    
    def runge_kutta_stepsize_first_order(h):
        def runge_kutta_function_first_order(t, x):
            k1 = h*f(t, x)
            k2 = h*f(t + h/2, x + k1/2)
            k3 = h*f(t + h/2, x + k2/2)
            k4 = h*f(t + h, x + k3)
            return x + 1/6*(k1 + 2*k2 + 2*k3 + k4)
        return runge_kutta_function_first_order
    return runge_kutta_stepsize_first_order

Next, we define a `solve` function, which takes in a constructor, an initial value `x_init`, a linear time array `tt`.

In [ ]:
def solve_first_order(constructor, x_init, tt):
    
    # extract data from time.
    n = len(tt)
    t_min, t_max = tt[0], tt[n-1]
    h = (t_max - t_min) / (n - 1)
    
    # construct function with stepsize h.
    d_function = constructor(h)
    
    # create solution sequence.
    solution = [None for _ in range(n)]
    
    # build solutions.
    for i, t in enumerate(tt):
        if i == 0:
            solution[0] = x_init
            pass
        else:
            solution[i] = d_function(t, solution[i - 1])
            pass
        pass
    return solution

A comparison of the first-order, second-order Euler methods, and the RK4 method.

In [ ]:
tt = np.linspace(0, 1, 10)
x_init = 2

euler_1 = solve_first_order(
    euler_method_constructor_first_order([lambda t, x:-np.power(x, 2)]), x_init, tt)
euler_2 = solve_first_order(
    euler_method_constructor_first_order([lambda t, x:-np.power(x, 2), lambda t, x:2*np.power(x, 3)]), x_init, tt)

rk = solve_first_order(
    runge_kutta_constructor_first_order(lambda t, x:-np.power(x, 2)), x_init, tt)
exact = x_init/(1 + x_init*tt)

for i, sol in enumerate(zip([euler_1, euler_2, rk, exact],
                            ['E1', 'E2', 'RK', 'exact'])):
    if i == 3:
        plt.scatter(tt, sol[0], label = '{}'.format(sol[1]), marker = 'x')
        pass
    elif i == 4:
        plt.scatter(tt, sol[0], label = '{}'.format(sol[1]))
    else:
        plt.plot(tt, sol[0], label = '{}'.format(sol[1]))
        pass
    pass
plt.title('Approximation Method Comparison')
plt.legend()
plt.show()

## Second Order ODE

We consider the nonlinear, second-order ODE: $\ddot{x} = -x^2$ with initial conditions $x_0 = 1$ and $\dot{x}_0 = 1$. The solution to this ODE is nasty, and involves some ugly Weierstrass elliptic function, which we will not inspect.

We can convert the above to a system of equations. Let $\dot{x} = y$, then $\dot{y} = \ddot{x} = -x^2$. Furthermore, $y_0 = 1$.

In [ ]:
# basic algorithm.

# set initial conditions.
x_init, y_init = 1, 1

# construct time set.
n = 10000
t_min = 0
t_max = 4
dt = (t_max - t_min) / (n - 1)
tt = np.linspace(t_min, t_max, n)

# construct solution sets.
xx = [None for _ in range(n)]
yy = [None for _ in range(n)]

# solve for solutions.
for i in range(n):
    if i == 0:
        xx[0] = x_init
        yy[0] = y_init
        pass
    else:
        xx[i] = xx[i - 1] + yy[i - 1]*dt
        yy[i] = yy[i - 1] - xx[i - 1]**2*dt
        pass
    pass

plt.plot(tt, xx)
plt.plot(tt, yy)
plt.title('$\ddot{x} = -x^2$')
plt.show()

Similarly, the system could be solved using higher-order methods as well as Runge-Kutta.

### Second Order Method Constructors

We write second-order method constructors for both the RK and Euler's method. This time, we have two functions of two variables for our first-order euler method.

In [ ]:
def euler_method_constructor(ff):
    # first order euler method.
    # ff: two functions of three variables (t, x, y).
    fx, fy = ff
    
    def euler_method_stepsize(h):
        
        def euler_method_function(t, x, y):
            x_ = x + fx(t, x, y)*h
            y_ = y + fy(t, x, y)*h
            return x_, y_
        
        return euler_method_function
    return euler_method_stepsize

def runge_kutta_constructor(ff):
    # runge-kutta method for second order system.
    # ff: two functions of three variables (t, x, y).
    fx, fy = ff
    
    def runge_kutta_stepsize(h):
        
        def runge_kutta_function(t, x, y):
            kx1 = h*fx(t, x, y)
            ky1 = h*fy(t, x, y)
            
            kx2 = h*fx(t + h/2, x + kx1/2, y + ky1/2)
            ky2 = h*fy(t + h/2, x + kx1/2, y + ky1/2)
            
            kx3 = h*fx(t + h/2, x + kx2/2, y + ky2/2)
            ky3 = h*fy(t + h/2, x + kx2/2, y + ky2/2)
            
            kx4 = h*fx(t + h, x + kx3, y + ky3)
            ky4 = h*fy(t + h, x + kx3, y + ky3)
            
            x_ = x + 1/6*(kx1 + 2*kx2 + 2*kx3 + kx4)
            y_ = y + 1/6*(ky1 + 2*ky2 + 2*ky3 + ky4)
            
            return x_, y_
        return runge_kutta_function
    return runge_kutta_stepsize

# initial conditions, time set.
def solve(constructor, initial_condition, tt):
    # extract data from tt.
    n = len(tt)
    t_min, t_max = tt[0], tt[n-1]
    h = (t_max - t_min) / (n - 1)
    
    # solve for stepsize function.
    method_function = constructor(h)
    
    # create solution sets.
    xx = [None for _ in range(n)]
    yy = [None for _ in range(n)]
    
    # solve.
    for i, t in enumerate(tt):
        if i == 0:
            xx[0], yy[0] = initial_condition
            pass
        else:
            xx[i], yy[i] = method_function(t, xx[i-1], yy[i-1])
            pass
        pass
    return xx, yy

Solving the previous system using the constructor.

In [ ]:
constructor = euler_method_constructor([lambda t, x, y: y, lambda t, x, y: -x**2])
initial_condition = [1, 1]
tt = np.linspace(0, 4, 10000)

xx, yy = solve(constructor, initial_condition, tt)
plt.plot(tt, xx)
plt.plot(tt, yy)
plt.title('$\ddot{x} = -x^2$')
plt.show()

We see that the result is the same.

In [ ]:
from mpl_toolkits import mplot3d

### Gravity

A test particle with mass $\mu$ is subject to the gravitational attraction of a (fixed) source particle with mass $M$ centered at the origin. The governing law for the motion of this particle is $\mu\ddot{\mathbf{r}} = -\dfrac{G\mu M}{r^2}\hat{\mathbf{r}}$, where $\hat{\mathbf{r}} = \dfrac{\mathbf{r}}{r}$. We convert this into a system. Let $\mathbf{s} = \dot{\mathbf{r}}$, then $\dot{\mathbf{s}} = -\dfrac{G\mu M}{r^2}\hat{\mathbf{r}}$. For simplicity, we solve the system $\dot{\mathbf{r}} = \mathbf{s} = \mathbf{f}(t, \mathbf{r}, \mathbf{s})$, $\dot{\mathbf{s}} = -\dfrac{2\mathbf{r}}{r^{3/2}} = \mathbf{g}(t, \mathbf{r}, \mathbf{s})$ (The $2$ in $\mathbf{g}$ is just there to make the system more interesting).

In [ ]:
# define relevant 'gravity' functions.

def f(t, r, s):
    # t is a scalar.
    # r, s are numpy arrays of length 2.
    return s

def g(t, r, s):
    r_norm = np.sqrt(r.dot(r))
    return -2/np.power(r_norm, 3/2) * r

We model a very simple system, $\mathbf{r}_0 = (1, 0)$ and $\mathbf{s}_0 = (0, 1)$. In this system, we predict that the test particle will move upwards, before the source particle's influence will drag the test particle towards it. In terms of parameters, $r_x$ decreases while $r_y$ increases, but $s_x$ becomes negative while $s_y$ is positive and decreasing. Then, the particle will probably go in some kind of orbit.

Simulating gravity with Euler's method.

In [ ]:
# initial condition.
constructor = euler_method_constructor([f, g])
initial_condition = [np.array([1, 0]), np.array([0, 1])]
tt = np.linspace(0, 100, 1000)

# we expect solutions r and s to be vectors.
rr, ss = solve(constructor, initial_condition, tt)

# extract position coordinates.
xx = [r[0] for r in rr]
yy = [r[1] for r in rr]

Plotting the system in various ways.

In [ ]:
plt.plot(tt, xx, label = 'x')
plt.plot(tt, yy, label = 'y')
plt.legend()
plt.title('Particle Coordinate Trajectories')
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
plt.plot(xx, yy, '--')
plt.scatter(0, 0, label = 'source')
plt.title('Particle Trajectory in Space')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize = (10, 10))
ax = plt.axes(projection='3d')
ax.plot3D(xx, yy, tt)
ax.view_init(10, 35)
plt.title('Particle Trajectory in Time')
plt.show()

Simulating gravity with Runge-Kutta method. (same conditions)

In [ ]:
# initial condition.
constructor = runge_kutta_constructor([f, g])
initial_condition = [np.array([1, 0]), np.array([0, 1])]
tt = np.linspace(0, 100, 1000)

# we expect solutions r and s to be vectors.
rr, ss = solve(constructor, initial_condition, tt)

# extract position coordinates.
xx = [r[0] for r in rr]
yy = [r[1] for r in rr]

In [ ]:
plt.plot(tt, xx, label = 'x')
plt.plot(tt, yy, label = 'y')
plt.legend()
plt.title('Particle Coordinate Trajectories')
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
plt.plot(xx, yy, '--')
plt.scatter(0, 0, label = 'source')
plt.title('Particle Trajectory in Space')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize = (10, 10))
ax = plt.axes(projection='3d')
ax.plot3D(xx, yy, tt)
ax.view_init(10, 35)
plt.title('Particle Trajectory in Time')
plt.show()

We see that the RK method is more accurate than Euler's method, when the timesteps are the same.